In [17]:
import requests
from bs4 import BeautifulSoup
import csv

url = "https://www.amazon.in/s?k=bags&crid=2M096C6104MLT&qid=1653308124&sprefix=ba%2Caps+C283&ref=sr_pg_1"

product_urls = []

def scrape_page_data(page_url):
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, "html.parser")

    products = soup.find_all("div", {"data-component-type": "s-search-result"})

    for product in products:
        product_url = "https://www.amazon.in" + product.find("a", {"class": "a-link-normal"})['href']
        
        if product_url:
            product_urls.append(product_url)

def scrape_multiple_pages(num_pages):
    for page in range(1, num_pages + 1):
        page_url = url.replace("sr_pg_1", f"sr_pg_{page}")
        scrape_page_data(page_url)
        
        if len(product_urls) >= 200:
            break

scrape_multiple_pages(20)

def scrape_product_data(product_url):
    response = requests.get(product_url)
    soup = BeautifulSoup(response.content, "html.parser")

    description = soup.find("div", {"id": "feature-bullets"})
    asin = soup.find("th", string="ASIN")
    product_description = soup.find("div", {"id": "productDescription"})
    manufacturer = soup.find("a", {"id": "bylineInfo"})

    description_text = description.get_text(strip=True) if description else ""
    asin_text = asin.find_next_sibling("td").get_text(strip=True) if asin else ""
    product_description_text = product_description.get_text(strip=True) if product_description else ""
    manufacturer_text = manufacturer.get_text(strip=True) if manufacturer else ""

    return {
        "Product URL": product_url,
        "Description": description_text,
        "ASIN": asin_text,
        "Product Description": product_description_text,
        "Manufacturer": manufacturer_text
    }

product_data = []
for url in product_urls:
    product_info = scrape_product_data(url)
    product_data.append(product_info)

filename = 'amazon_products_extended.csv'
headers = ['Product URL', 'Description', 'ASIN', 'Product Description', 'Manufacturer']

with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers)
    writer.writeheader()
    writer.writerows(product_data)

print(f'Data successfully written to {filename}.')


Data successfully written to amazon_products_extended1.csv.
